In [1]:
import requests
import json
import requests
import pandas as pd
from plotly import graph_objects as go
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
%run roadspending_gdp.ipynb
%run trips_duration.ipynb

Note: you may need to restart the kernel to use updated packages.


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [ ]:
def merging_data(final_gdp_roadspending_df, final_1plus_df):

    tourist_num = final_1plus_df
    tourist_num_copy = final_1plus_df.copy()
    gdp = final_gdp_roadspending_df[final_gdp_roadspending_df['Year'] > 2011]
    gdp_copy = final_gdp_roadspending_df[final_gdp_roadspending_df['Year'] > 2011].copy()


    min_trips = tourist_num['NumTrips'].min()
    max_trips = tourist_num['NumTrips'].max()

    min_gdp = gdp['GDP'].min()
    max_gdp = gdp['GDP'].max()

    tourist_num['Tourism St.'] = (tourist_num['NumTrips'] - min_trips) / (max_trips - min_trips)
    gdp['GDP St.'] = (gdp['GDP'] - min_gdp) / (max_gdp - min_gdp)

    tourist_num = pd.DataFrame(final_1plus_df, columns = ['Country', 'Year', 'Tourism St.'])
    gdp_actual = pd.DataFrame(gdp, columns = ["Country", "Year", "GDP St."])

    prediction_data = pd.merge(tourist_num, gdp_actual, on = ['Country', 'Year'])
    prediction_data["Year"] = prediction_data['Year'].astype(int)

    unfiltered_predicted_gdp =pd.DataFrame(gdp_copy, columns = ["Country", "Year", "GDP"])
    unfiltered_predicted_tourist =pd.DataFrame(tourist_num_copy, columns = ["Country", "Year", "NumTrips"])
    unfiltered_predicted_data = pd.merge(unfiltered_predicted_gdp, unfiltered_predicted_tourist, on = ['Country', 'Year'])

    return prediction_data, unfiltered_predicted_data




In [ ]:
datas = merging_data(final_gdp_roadspending_df, final_1plus_df)
data = datas[0]
data_not_st = datas[1]
data_not_st

/var/folders/6f/26l7p7y90rd4sw3bc17185400000gn/T/ipykernel_80612/4017440539.py:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Country,Year,GDP,NumTrips
0,Austria,2012,3.645983e+11,36003256
1,Austria,2013,3.828753e+11,36479782
2,Austria,2014,3.940021e+11,36681593
3,Austria,2015,3.399889e+11,35815124
4,Austria,2016,3.521979e+11,39365940
...,...,...,...,...
292,Sweden,2018,4.906677e+11,89740946
293,Sweden,2019,4.736304e+11,89854108
294,Sweden,2020,4.851817e+11,98845718
295,Sweden,2021,5.670964e+11,105262786


In [ ]:
def get_lag_columns(country_df, lag = 3):

    country_df = country_df.sort_values('Year').reset_index(drop = True)
    country_df['Covid'] = country_df['Year'].apply(lambda x: 1 if x in [2020, 2021] else 0)
    country_df['GDP_lag'] = country_df['GDP St.'].shift(lag)
    country_df['Tourism_lag'] = country_df['Tourism St.'].shift(lag)
    country_df['Bias'] = 1

    return country_df.dropna().reset_index(drop = True)

In [ ]:
def fit_regression(X, y):
    Xt = X.T
    XtX = np.dot(Xt, X)
    XtX_inv = np.linalg.pinv(XtX)
    Xt_y = np.dot(Xt, y)
    beta = np.dot(XtX_inv, Xt_y)

    return beta

In [ ]:
def predict_next_year(beta, X_new):
    return np.dot(beta, X_new)

In [ ]:

def get_country_years_pred(data, country, lag = 3):
    country_df0 = data[data['Country'] == country]
    country_df = get_lag_columns(country_df0, lag = lag)
    
    X = country_df[['Bias', 'GDP_lag', 'Tourism_lag']].values
    y_gdp = country_df['GDP St.'].values
    y_tourism = country_df['Tourism St.'].values

    beta_gdp = fit_regression(X, y_gdp)
    beta_tourism = fit_regression(X, y_tourism)

    predictions = []
    for idx, row in country_df.iterrows():
        year = row['Year']
        X_year = np.array([row['Bias'], row['GDP_lag'], row['Tourism_lag']])
        pred_gdp = predict_next_year(beta_gdp, X_year)
        pred_tourism = predict_next_year(beta_tourism, X_year)

        predictions.append({
            'Country': country,
            'Year': year,
            'Actual_GDP': row['GDP St.'],
            'Predicted_GDP': pred_gdp,
            'Actual_Tourism': row['Tourism St.'],
            'Predicted_Tourism': pred_tourism

        })

    all_years_results = pd.DataFrame(predictions)

    return all_years_results

In [ ]:
prev_year_result = get_country_years_pred(data, "Spain", lag=3)
# print(prev_year_result)


In [ ]:

def get_country_prediction(data, country, lag=3):
    country_df0 = data[data['Country'] == country].copy()
    country_df = get_lag_columns(country_df0, lag=lag)

    X = country_df[['Bias', 'GDP_lag', 'Tourism_lag']].values
    y_gdp = country_df['GDP St.'].values
    y_tourism = country_df['Tourism St.'].values

    beta_gdp = fit_regression(X, y_gdp)
    beta_tourism = fit_regression(X, y_tourism)

    # Get the most recent year in the dataset
    last_year = country_df['Year'].max()

    # Get initial lag values
    lag_year = last_year - lag + 1
    lag_row = country_df[country_df['Year'] == lag_year]
    
    gdp_lag_val = lag_row['GDP St.'].values[0]
    tourism_lag_val = lag_row['Tourism St.'].values[0]

    # Store predictions for each future year
    predictions = []

    for i in range(1, lag + 1):
        year = last_year + i
        X_new = np.array([1, gdp_lag_val, tourism_lag_val])
        
        pred_gdp = predict_next_year(beta_gdp, X_new)
        pred_tourism = predict_next_year(beta_tourism, X_new)

        predictions.append({
            'Year': year,
            'Country': country,
            'Predicted_GDP': pred_gdp,
            'Predicted_Tourism': pred_tourism
        })

        # Update lag values for next step
        gdp_lag_val = pred_gdp
        tourism_lag_val = pred_tourism

    return predictions


In [ ]:
result = get_country_prediction(data, "Slovenia", lag=3)
result = pd.DataFrame(result)
result.sort_values(by=['Year'])
result

,Year,Country,Predicted_GDP,Predicted_Tourism
0,2024,Slovenia,0.010373,0.016913
1,2025,Slovenia,0.010743,0.020441
2,2026,Slovenia,0.009787,0.014855


In [ ]:
def visualization(data, st_data): 
    min_trips = data['NumTrips'].min()
    max_trips = data['NumTrips'].max()

    min_gdp = data['GDP'].min()
    max_gdp = data['GDP'].max()

    st_data["Predicted_GDP_normal"] = st_data["Predicted_GDP"] * (max_gdp - min_gdp) + min_gdp
    st_data["Predicted_Tourism_normal"] = st_data["Predicted_Tourism"] * (max_trips - min_trips) + min_trips

    
    country_name = st_data["Country"].iloc[0]
    matching_data = data[data["Country"] == country_name]

    merged = pd.concat([st_data, matching_data], ignore_index=True)
    merged = merged.sort_values(by="Year").reset_index(drop=True)

    merged = merged.drop(columns = ["Predicted_GDP", "Predicted_Tourism"])

    merged['GDP_combined'] = merged['GDP'].fillna(merged['Predicted_GDP_normal'])
    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])
    merged['GDP_combined'] = merged['GDP'].fillna(merged['Predicted_GDP_normal'])
    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])

    merged['GDP_combined'] = merged['GDP'].fillna(merged['Predicted_GDP_normal'])
    merged['Tourism_combined'] = merged['NumTrips'].fillna(merged['Predicted_Tourism_normal'])
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=merged['Year'], 
        y=merged['GDP_combined'], 
        name='GDP',
        line=dict(color='firebrick', width=4),
        yaxis='y'
    ))

# Tourism trace (right Y-axis)
    fig.add_trace(go.Scatter(
        x=merged['Year'], 
        y=merged['Tourism_combined'], 
        name='Tourist Numbers',
        line=dict(color='royalblue', width=4),
        yaxis='y2'
    ))

    fig.update_layout(
        title=f'Predicted GDP and Tourism Numbers for {country_name}',
        xaxis=dict(title="Years"),
        yaxis=dict(
            title="GDP",
            titlefont=dict(color="firebrick"),
            tickfont=dict(color="firebrick")
        ),
        yaxis2=dict(
            title="Tourism Numbers",
            titlefont=dict(color="royalblue"),
            tickfont=dict(color="royalblue"),
            overlaying="y",
            side="right"
        ),
        legend=dict(x=0.003, y=0.99),
    )

    fig.add_vline(
        x=2023,  
        line=dict(color='gray', dash='dot'),
        annotation_text='Prediction',
        annotation_position='top right'
)

    fig.show()


In [ ]:
visualization(data_not_st, result)

In [ ]:
def cross_validate_time_series(country_df, lag = 3):
    results = []
    country_df = country_df.sort_values('Year').reset_index(drop = True)

    for i in range(lag, len(country_df) - 1):
        train_df = country_df.iloc[:i]
        test_df = country_df.iloc[i:i+1]

        X_train = train_df[['Bias', 'GDP_lag', 'Tourism_lag']].values
        y_gdp_train = train_df['GDP St.'].values
        y_tourism_train = train_df['Tourism St.'].values

        X_test = test_df[['Bias', 'GDP_lag', 'Tourism_lag']].values[0]

        beta_gdp = fit_regression(X_train, y_gdp_train)
        beta_tourism  = fit_regression(X_train, y_tourism_train)

        pred_gdp = predict_next_year(beta_gdp, X_test)
        pred_tourism = predict_next_year(beta_tourism, X_test)

        results.append({
            'Year': test_df['Year'].values[0],
            'Actual_GDP': test_df['GDP St.'].values[0],
            'Predicted_GDP': pred_gdp,
            'Actual_Tourism': test_df['Tourism St.'].values[0],
            'Predicted_Tourism': pred_tourism
        })

    predictions = pd.DataFrame(results)

    return predictions

In [ ]:
def evaluate_predictions(pred_df):
    r2_gdp = r2_score(pred_df['Actual_GDP'], pred_df['Predicted_GDP'])
    mse_gdp = mean_squared_error(pred_df['Actual_GDP'], pred_df['Predicted_GDP'])

    r2_tourism = r2_score(pred_df['Actual_Tourism'], pred_df['Predicted_Tourism'])
    mse_tourism = mean_squared_error(pred_df['Actual_Tourism'], pred_df['Predicted_Tourism'])

    evaluations = {
        'GDP': {'R2': r2_gdp, 'MSE': mse_gdp},
        'Tourism': {'R2': r2_tourism, 'MSE': mse_tourism}
    }

    return evaluations

In [ ]:
EU_countries_list = [
    'Austria', "Belgium", "Bulgaria", "Croatia", "Czechia",
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary",
    "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta",
    "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia",
    "Spain", "Sweden"]

for country in EU_countries_list:
    country_df = data[data['Country'] == country].copy()
    country_df = get_lag_columns(country_df, lag = 3)
    # country_df

    pred_df = cross_validate_time_series(country_df, lag = 3)
    metrics = evaluate_predictions(pred_df)

    #print(pred_df)
    print(country)
    print(metrics)

Austria
{'GDP': {'R2': -526.6283685281678, 'MSE': 0.007090018443050572}, 'Tourism': {'R2': -7.6320286345900445, 'MSE': 0.003786873631911201}}
Belgium
{'GDP': {'R2': -8.410621811985504, 'MSE': 0.0003966878083569504}, 'Tourism': {'R2': -0.1746415116511506, 'MSE': 0.00024643516195115}}
Bulgaria
{'GDP': {'R2': -7.6289373866986985, 'MSE': 3.7948734582065155e-05}, 'Tourism': {'R2': -9.89687707996579, 'MSE': 0.00014568583773363178}}
Croatia
{'GDP': {'R2': -3.5306361081777125, 'MSE': 6.088035334546693e-06}, 'Tourism': {'R2': -21.36737236737935, 'MSE': 0.0001698819445137793}}
Czechia
{'GDP': {'R2': -884.3110865129993, 'MSE': 0.020018156374552665}, 'Tourism': {'R2': -593.9296952318911, 'MSE': 0.1685823568352502}}
Denmark
{'GDP': {'R2': -42.55055920970013, 'MSE': 0.001479283263745057}, 'Tourism': {'R2': -69.48692763128646, 'MSE': 0.01508057149503358}}
Estonia
{'GDP': {'R2': -1.3465762137224884, 'MSE': 1.0743557678679102e-06}, 'Tourism': {'R2': -1.3959476252670888, 'MSE': 4.132220323873984e-05}}
F

Lithuania
{'GDP': {'R2': -22.5586684193488, 'MSE': 5.6312787890009005e-05}, 'Tourism': {'R2': -0.7449188518266734, 'MSE': 1.6803328638618243e-05}}
Luxembourg
{'GDP': {'R2': -3.5344181764946647, 'MSE': 8.543476537370785e-06}, 'Tourism': {'R2': -2.074856840910267, 'MSE': 2.3265246454922207e-05}}
Malta
{'GDP': {'R2': -6.4536326358846505, 'MSE': 9.017470198829393e-07}, 'Tourism': {'R2': -2.240667615953502, 'MSE': 1.5252452265040858e-06}}
Netherlands
{'GDP': {'R2': -58.62923867470188, 'MSE': 0.010278080091507595}, 'Tourism': {'R2': -1.4977346501116768, 'MSE': 0.002053080310695207}}
Poland
{'GDP': {'R2': -4.822752053638071, 'MSE': 0.0005714349453288712}, 'Tourism': {'R2': -3.3983916711985245, 'MSE': 0.002948749604215448}}
Portugal
{'GDP': {'R2': -24.837218418675196, 'MSE': 0.00013008397162450096}, 'Tourism': {'R2': -5.158902016831685, 'MSE': 0.0006217510176721979}}
Romania
{'GDP': {'R2': -3.500032838628557, 'MSE': 0.00010833279300512182}, 'Tourism': {'R2': -2.922421506246286, 'MSE': 0.000403

KeyError: 'Actual_GDP'

[ 1.38460300e-05  1.60803542e+00 -4.02768646e-01]
[-1.96379045e-03  2.15495521e+00 -1.42390863e-01]
[ 0.03149784  0.04841447 -1.66681355]
[ 0.03746867  0.20159158 -1.72555716]
[ 0.02790968 -0.76927567 -0.92408123]
[ 0.03615263 -0.09831528 -1.45314282]
[ 0.01205886 -1.15677074  0.4314224 ]
[ 0.06514452  0.61043175 -3.93242386]
[-0.00532577 -0.74594642  1.42539223]
[ 1.93327579e-03  2.10420564e+00 -3.18309220e-01]
   Year  Actual_GDP  Predicted_GDP  Actual_Tourism  Predicted_Tourism
0  2018    0.009762       0.006371        0.016154           0.011907
1  2019    0.009807       0.008238        0.015126           0.014551
2  2020    0.009679       0.007431        0.009315           0.013427
3  2021    0.011501       0.007736        0.021268           0.007579
4  2022    0.011148       0.008920        0.024754           0.017753
{'GDP': {'R2': -11.516155940270366, 'MSE': 7.62958075236341e-06}, 'Tourism': {'R2': -0.9252703594158413, 'MSE': 5.43325441566272e-05}}


In [ ]:
X = country_df[['Bias', 'GDP_lag', 'Tourism_lag']].values
y_gdp = country_df['GDP St.'].values
y_tourism = country_df['Tourism St.'].values
y_gdp

array([0.1002275 , 0.10322657, 0.10912732, 0.11819428, 0.11650724,
       0.11422163, 0.13098019, 0.1271656 , 0.13786336])